# Lab 4: AgentCore Runtime으로 프로덕션 배포

## 개요

이전 랩들에서 메모리, 공유 도구, 프로덕션급 배포를 갖춘 포괄적인 이커머스 고객 지원 에이전트를 구축했습니다. AgentCore 서비스의 기능을 보여주어 에이전트 사용 사례를 프로토타입에서 프로덕션으로 마이그레이션하는 과정을 시연했습니다. 이제 실제 고객 대화를 처리할 수 있는 시스템이 준비되었습니다.

[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html)은 프레임워크, 프로토콜 또는 모델 선택에 관계없이 조직이 프로덕션에서 AI 에이전트를 배포하고 확장할 수 있도록 지원하는 안전하고 완전히 관리되는 런타임입니다. 엔터프라이즈급 안정성, 자동 확장 및 포괄적인 모니터링 기능을 제공합니다.

**워크숍 여정:**

- **Lab 1 (완료)**: 에이전트 프로토타입 생성 - 기능적인 이커머스 고객 지원 에이전트 구축
- **Lab 2 (완료)**: 메모리로 강화 - 대화 컨텍스트 및 개인화 추가
- **Lab 3 (완료)**: Gateway & Identity로 확장 - 에이전트 간 도구 안전하게 공유
- **Lab 4 (현재)**: 프로덕션 배포 - AgentCore Runtime으로 관측성 확보
- **Lab 5**: 사용자 인터페이스 구축 - 고객 대상 애플리케이션 생성

### AgentCore Runtime & 프로덕션 배포의 중요성

현재 상태 (Lab 1-3): 에이전트가 로컬에서 중앙 집중식 도구로 실행되지만 프로덕션 과제에 직면:

- 에이전트가 단일 세션에서 로컬로 실행됨
- 포괄적인 모니터링 또는 디버깅 기능 없음
- 여러 동시 사용자를 안정적으로 처리할 수 없음

이 랩 후에는 다음과 같은 프로덕션 준비 에이전트 인프라를 갖게 됩니다:

- 가변 수요를 처리하는 서버리스 자동 확장
- 추적, 메트릭 및 로깅을 통한 포괄적인 관측성
- 자동 오류 복구를 통한 엔터프라이즈 안정성
- 적절한 액세스 제어를 통한 보안 배포
- AWS 콘솔 및 API를 통한 쉬운 관리 및 실제 프로덕션 워크로드 지원

### AgentCore 관측성을 통한 포괄적인 관측성 추가

또한 AgentCore Runtime은 [AgentCore 관측성](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html)과 원활하게 통합되어 프로덕션에서 에이전트의 동작에 대한 완전한 가시성을 제공합니다. AgentCore 관측성은 에이전트 상호작용, 도구 사용 및 메모리 액세스 패턴에서 추적, 메트릭 및 로그를 자동으로 캡처합니다. 이 랩에서는 AgentCore Runtime이 CloudWatch GenAI 관측성과 통합되어 포괄적인 모니터링 및 디버깅 기능을 제공하는 방법을 살펴보겠습니다.

요청 추적의 경우 AgentCore 관측성은 도구 호출, 메모리 검색 및 모델 상호작용을 포함한 완전한 대화 흐름을 캡처합니다. 성능 모니터링의 경우 응답 시간, 성공률 및 리소스 사용률을 추적하여 에이전트의 성능을 최적화하는 데 도움이 됩니다.

관측성 흐름 중에 AgentCore Runtime은 에이전트 코드를 자동으로 계측하고 CloudWatch에 원격 측정 데이터를 보냅니다. 그런 다음 CloudWatch 대시보드 및 GenAI 관측성 기능을 사용하여 패턴을 분석하고, 병목 현상을 식별하고, 실시간으로 문제를 해결할 수 있습니다.

### Lab 4를 위한 아키텍처
<div style="text-align:left"> 
    <img src="images/architecture_lab4_runtime.png" width="75%"/> 
</div>

*CloudWatch를 통한 완전한 관측성을 갖춘 AgentCore Runtime에서 실행되는 에이전트로, 자동 확장 및 포괄적인 모니터링을 통해 프로덕션 트래픽을 처리합니다. 이전 랩의 메모리 및 게이트웨이 통합은 프로덕션 환경에서 완전히 기능합니다.*

### 주요 기능

- **서버리스 에이전트 배포:** 최소한의 코드 변경으로 로컬 에이전트를 확장 가능한 프로덕션 서비스로 변환
- **포괄적인 관측성:** CloudWatch GenAI 관측성을 통한 완전한 요청 추적, 성능 메트릭 및 디버깅 기능

### 전제 조건

- Python 3.12+
- 적절한 권한이 있는 AWS 계정
- Docker, Finch 또는 Podman 설치 및 실행
- Amazon Bedrock AgentCore SDK
- Strands Agents 프레임워크

**참고**: AgentCore 관측성 추적을 CloudWatch에서 보려면 [CloudWatch Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html)를 활성화해야 합니다.

### Step 1: 필수 라이브러리 가져오기

In [ ]:
# 필수 라이브러리 가져오기
import os
import json
import boto3
from strands import Agent
from strands.models import BedrockModel
import sys; sys.path.insert(0, '..'); from src.helpers.ecommerce_memory import create_or_get_ecommerce_memory_resource

create_or_get_ecommerce_memory_resource()  # 메모리 랩이 실행되지 않은 경우를 대비

### Step 2: AgentCore Runtime용 에이전트 준비

#### Runtime 호환 에이전트 생성

Python SDK를 통해 이전 로컬 에이전트 구현 내에서 필요한 AgentCore Runtime 구성 요소를 먼저 정의해보겠습니다.

아래 `#### AGENTCORE RUNTIME - LINE i ####` 주석을 보면 런타임 준비 에이전트를 준비하는 관련 배포 코드가 추가된 위치를 확인할 수 있습니다. 런타임 준비 에이전트를 준비하는 4개의 라인을 찾을 수 있습니다:

1. `from bedrock_agentcore.runtime import BedrockAgentCoreApp`으로 Runtime App 가져오기
2. `app = BedrockAgentCoreApp()`으로 App 초기화
3. `@app.entrypoint`로 호출 함수 데코레이트
4. `app.run()`으로 AgentCore Runtime이 실행을 제어하도록 함

In [ ]:
%%writefile ../src/helpers/lab4_runtime.py
from bedrock_agentcore.runtime import (
    BedrockAgentCoreApp,
)  #### AGENTCORE RUNTIME - LINE 1 ####
from strands import Agent
from strands.models import BedrockModel
from src.helpers.lab1_strands_agent import (
    check_return_eligibility,
    process_return_request,
    get_product_recommendations,
    ECOMMERCE_SYSTEM_PROMPT,
    ECOMMERCE_MODEL_ID,
)

# Lab1 import: Bedrock 모델 생성
model = BedrockModel(model_id=ECOMMERCE_MODEL_ID)

# 이커머스 고객 지원 도구가 포함된 에이전트 생성 (메모리 없이)
agent = Agent(
    model=model,
    tools=[check_return_eligibility, process_return_request, get_product_recommendations],
    system_prompt=ECOMMERCE_SYSTEM_PROMPT,
)

# AgentCore Runtime App 초기화
app = BedrockAgentCoreApp()  #### AGENTCORE RUNTIME - LINE 2 ####


@app.entrypoint  #### AGENTCORE RUNTIME - LINE 3 ####
def invoke(payload):
    """AgentCore Runtime 엔트리포인트 함수"""
    user_input = payload.get("prompt", "")

    # 에이전트 호출
    response = agent(user_input)
    return response.message["content"][0]["text"]


if __name__ == "__main__":
    app.run()  #### AGENTCORE RUNTIME - LINE 4 ####

### Step 3: AgentCore Runtime에 배포

이제 [AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit)을 사용하여 에이전트를 AgentCore Runtime에 배포하겠습니다.

#### 보안 Runtime 배포 구성 (AgentCore Runtime + AgentCore Identity)

먼저 스타터 툴킷을 사용하여 엔트리포인트, 실행 역할, requirements 파일과 함께 AgentCore Runtime 배포를 구성합니다. 또한 Amazon Cognito 사용자 풀을 사용하여 신원 인증을 구성하고 시작 시 Amazon ECR 리포지토리를 자동 생성하도록 스타터 킷을 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 Dockerfile이 생성됩니다.

**참고**: Cognito access_token은 2시간만 유효합니다. access_token이 만료되면 `reauthenticate_user` 메서드를 사용하여 새 access_token을 발급받을 수 있습니다.

In [ ]:
from src.helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Amazon Cognito 사용자 풀 재설정 중...")

# 기존 Cognito 설정 삭제 (있다면)
import boto3
cognito_client = boto3.client('cognito-idp')

# 새로운 Cognito 설정 생성
cognito_config = setup_cognito_user_pool()  # Bearer 토큰은 이 출력 셀에서 확인하세요
print("Cognito 재설정 완료 ✓")
print(f"새 Client ID: {cognito_config['client_id']}")
print(f"새 Bearer Token 길이: {len(cognito_config['bearer_token'])}")

### Step 4: Runtime 세션으로 에이전트 테스트

이제 AgentCore Runtime에서 실행되는 이커머스 에이전트를 테스트해보겠습니다.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from src.helpers.utils import create_agentcore_runtime_execution_role

# 기존 설정 파일 삭제
import os
if os.path.exists(".bedrock_agentcore.yaml"):
    os.remove(".bedrock_agentcore.yaml")
    print("기존 설정 파일 삭제 완료")

# Runtime 툴킷 초기화
boto_session = boto3.session.Session()
region = boto_session.region_name

execution_role_arn = create_agentcore_runtime_execution_role()

# 완전히 새로운 Runtime 인스턴스 생성
agentcore_runtime = Runtime()

# 새로운 에이전트 이름으로 배포 구성 (Cognito 인증 포함)
import uuid
agent_suffix = str(uuid.uuid4())[:8]
agent_name = f"ecommerce_test_{agent_suffix}"

# Cognito 인증과 함께 구성
response = agentcore_runtime.configure(
    entrypoint="src/helpers/lab4_runtime.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
)

print("✅ Runtime 구성 완료 (Cognito 인증 포함)")
print(f"📋 새 에이전트 이름: {agent_name}")
print(f"🔐 Cognito Client ID: {cognito_config.get('client_id')}")
print(f"🌐 Discovery URL: {cognito_config.get('discovery_url')}")

### Step 5: 관측성 및 모니터링 설정

CloudWatch를 통한 포괄적인 관측성을 설정하겠습니다.

In [ ]:
# Import 에러 수정 후 새 에이전트 배포
from src.helpers.utils import put_ssm_parameter

print("Import 에러 수정 후 새 에이전트 배포 중...")
launch_result = agentcore_runtime.launch()
print("배포 완료:", launch_result.agent_arn)

agent_arn = put_ssm_parameter(
    "/app/ecommerce/agentcore/runtime_arn", launch_result.agent_arn
)

### Step 6: 프로덕션 준비 상태 확인

에이전트가 프로덕션 트래픽을 처리할 준비가 되었는지 확인하겠습니다.

In [ ]:
# 배포 완료까지 대기
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    print(f"배포 대기 중... 현재 상태: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]

print(f"최종 상태: {status}")

### Step 7: 성능 및 확장성 시뮬레이션

프로덕션 환경에서의 성능을 시뮬레이션해보겠습니다.

In [ ]:
# 세션 연속성 시연을 위한 세션 ID 생성
import uuid
import time
import sys
import json

session_id = uuid.uuid4()

# 이커머스 고객 지원 시나리오 테스트 (스트리밍 응답)
user_query = "지난주에 주문한 원피스 사이즈가 안 맞아서 반품하고 싶어요."

bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

print("🤖 K-Style 고객센터 에이전트가 답변 중입니다...")
print("-" * 60)

# 스트리밍 응답을 위한 invoke_streaming 사용
try:
    response_generator = agentcore_runtime.invoke_streaming(
        {"prompt": user_query}, 
        bearer_token=bearer_token,
        session_id=str(session_id)
    )
    
    print("✅ 에이전트 응답:")
    full_response = ""
    
    # 스트리밍 응답 처리
    for chunk in response_generator:
        if isinstance(chunk, dict) and 'response' in chunk:
            chunk_text = chunk['response']
            if isinstance(chunk_text, str):
                # JSON 문자열을 파싱하여 이스케이프 문자 처리
                try:
                    parsed_text = json.loads(chunk_text) if chunk_text.startswith('"') else chunk_text
                    print(parsed_text, end='', flush=True)
                    full_response += parsed_text
                except:
                    print(chunk_text, end='', flush=True)
                    full_response += chunk_text
                time.sleep(0.02)
        elif isinstance(chunk, str):
            print(chunk, end='', flush=True)
            full_response += chunk
            time.sleep(0.02)
    
    print("\n")
    print("-" * 60)
    print("✅ 응답 완료")
    
except AttributeError:
    # invoke_streaming이 없는 경우 일반 invoke 사용하되 타이핑 효과 시뮬레이션
    print("🔄 스트리밍 모드를 사용할 수 없어 일반 모드로 전환합니다...")
    
    response = agentcore_runtime.invoke(
        {"prompt": user_query}, 
        bearer_token=bearer_token,
        session_id=str(session_id)
    )
    
    # 응답 텍스트 추출 및 JSON 파싱
    if 'response' in response:
        if isinstance(response['response'], str):
            response_text = response['response']
            # JSON 문자열인 경우 파싱하여 이스케이프 문자 처리
            if response_text.startswith('"') and response_text.endswith('"'):
                try:
                    response_text = json.loads(response_text)
                except:
                    pass
        elif isinstance(response['response'], list) and len(response['response']) > 0:
            response_text = response['response'][0]
            if isinstance(response_text, bytes):
                response_text = response_text.decode('utf-8')
        else:
            response_text = str(response['response'])
    else:
        response_text = str(response)
    
    print("✅ 에이전트 응답:")
    print()  # 빈 줄 추가
    
    # 타이핑 효과로 출력 (이스케이프 문자는 실제 줄바꿈으로 처리)
    for char in response_text:
        if char == '\\':
            # 다음 문자 확인
            continue
        print(char, end='', flush=True)
        time.sleep(0.03)
    
    print("\n")
    print("-" * 60)
    print("✅ 응답 완료")

except Exception as e:
    print(f"❌ 에러 발생: {e}")
    response = agentcore_runtime.invoke(
        {"prompt": user_query}, 
        bearer_token=bearer_token,
        session_id=str(session_id)
    )
    
    # 간단한 출력 (JSON 파싱 포함)
    if 'response' in response and isinstance(response['response'], str):
        response_text = response['response']
        if response_text.startswith('"') and response_text.endswith('"'):
            try:
                response_text = json.loads(response_text)
            except:
                pass
        print("✅ 에이전트 응답:")
        print(response_text)
    else:
        print("✅ 에이전트 응답:")
        print(response)

#### 세션 연속성을 통한 에이전트 호출

AgentCore Runtime을 사용하고 있으므로 동일한 세션 ID로 대화를 쉽게 계속할 수 있습니다.

In [ ]:
# 세션 연속성 테스트 (스트리밍 출력)
import json
import time

user_query = "주문번호는 ORD-20240810-001이고, 상품명은 '플라워 패턴 원피스'입니다."

print("🔄 같은 세션에서 추가 정보 제공 중...")
print("-" * 60)

try:
    response_generator = agentcore_runtime.invoke_streaming(
        {"prompt": user_query}, 
        bearer_token=bearer_token,
        session_id=str(session_id)  # 같은 세션 ID 사용
    )
    
    print("✅ 세션 연속성 응답:")
    print()  # 빈 줄 추가
    
    # 스트리밍 응답 처리
    for chunk in response_generator:
        if isinstance(chunk, dict) and 'response' in chunk:
            chunk_text = chunk['response']
            if isinstance(chunk_text, str):
                try:
                    parsed_text = json.loads(chunk_text) if chunk_text.startswith('"') else chunk_text
                    print(parsed_text, end='', flush=True)
                except:
                    print(chunk_text, end='', flush=True)
                time.sleep(0.02)
        elif isinstance(chunk, str):
            print(chunk, end='', flush=True)
            time.sleep(0.02)
    
    print("\n")
    print("-" * 60)
    print("✅ 세션 연속성 확인 완료 - 이전 대화 맥락을 기억하고 있습니다!")
    
except AttributeError:
    # invoke_streaming이 없는 경우 일반 invoke 사용
    print("🔄 스트리밍 모드를 사용할 수 없어 일반 모드로 전환합니다...")
    
    response = agentcore_runtime.invoke(
        {"prompt": user_query}, 
        bearer_token=bearer_token,
        session_id=str(session_id)  # 같은 세션 ID 사용
    )
    
    # 응답 텍스트 추출 및 JSON 파싱
    if 'response' in response:
        if isinstance(response['response'], str):
            response_text = response['response']
            if response_text.startswith('"') and response_text.endswith('"'):
                try:
                    response_text = json.loads(response_text)
                except:
                    pass
        elif isinstance(response['response'], list) and len(response['response']) > 0:
            response_text = response['response'][0]
            if isinstance(response_text, bytes):
                response_text = response_text.decode('utf-8')
        else:
            response_text = str(response['response'])
    else:
        response_text = str(response)
    
    print("✅ 세션 연속성 응답:")
    print()  # 빈 줄 추가
    
    # 타이핑 효과로 출력
    for char in response_text:
        print(char, end='', flush=True)
        time.sleep(0.03)
    
    print("\n")
    print("-" * 60)
    print("✅ 세션 연속성 확인 완료 - 이전 대화 맥락을 기억하고 있습니다!")

except Exception as e:
    print(f"❌ 에러 발생: {e}")
    response = agentcore_runtime.invoke(
        {"prompt": user_query}, 
        bearer_token=bearer_token,
        session_id=str(session_id)
    )
    
    if 'response' in response and isinstance(response['response'], str):
        response_text = response['response']
        if response_text.startswith('"') and response_text.endswith('"'):
            try:
                response_text = json.loads(response_text)
            except:
                pass
        print("✅ 세션 연속성 응답:")
        print(response_text)
    else:
        print("✅ 세션 연속성 응답:")
        print(response)

#### 새로운 사용자와 에이전트 호출
아래 예시에서는 두 번째 쿼리에서 특정 제품을 언급하지 않았지만 에이전트는 여전히 그 맥락을 가지고 있습니다. 이는 AgentCore Runtime 세션 연속성 때문입니다. 에이전트는 새로운 사용자에 대한 맥락을 알지 못합니다.

In [ ]:
# 새로운 고객을 시연하기 위한 새 세션 ID 생성 (스트리밍 출력)
import json
import time

session_id2 = uuid.uuid4()

user_query = "여전히 작동하지 않아요. 무슨 일이 일어나고 있는 거죠?"

print("👤 새로운 고객 세션 시작...")
print("🤖 에이전트가 새로운 맥락에서 응답 중...")
print("-" * 60)

try:
    response_generator = agentcore_runtime.invoke_streaming(
        {"prompt": user_query}, 
        bearer_token=bearer_token,
        session_id=str(session_id2)  # 새로운 세션 ID 사용
    )
    
    print("✅ 새 고객 응답:")
    print()  # 빈 줄 추가
    
    # 스트리밍 응답 처리
    for chunk in response_generator:
        if isinstance(chunk, dict) and 'response' in chunk:
            chunk_text = chunk['response']
            if isinstance(chunk_text, str):
                try:
                    parsed_text = json.loads(chunk_text) if chunk_text.startswith('"') else chunk_text
                    print(parsed_text, end='', flush=True)
                except:
                    print(chunk_text, end='', flush=True)
                time.sleep(0.02)
        elif isinstance(chunk, str):
            print(chunk, end='', flush=True)
            time.sleep(0.02)
    
    print("\n")
    print("-" * 60)
    print("✅ 새 세션 확인 완료 - 이전 대화 맥락을 기억하지 않고 새로운 대화로 시작!")
    
except AttributeError:
    # invoke_streaming이 없는 경우 일반 invoke 사용
    print("🔄 스트리밍 모드를 사용할 수 없어 일반 모드로 전환합니다...")
    
    response = agentcore_runtime.invoke(
        {"prompt": user_query}, 
        bearer_token=bearer_token,
        session_id=str(session_id2)  # 새로운 세션 ID 사용
    )
    
    # 응답 텍스트 추출 및 JSON 파싱
    if 'response' in response:
        if isinstance(response['response'], str):
            response_text = response['response']
            if response_text.startswith('"') and response_text.endswith('"'):
                try:
                    response_text = json.loads(response_text)
                except:
                    pass
        elif isinstance(response['response'], list) and len(response['response']) > 0:
            response_text = response['response'][0]
            if isinstance(response_text, bytes):
                response_text = response_text.decode('utf-8')
        else:
            response_text = str(response['response'])
    else:
        response_text = str(response)
    
    print("✅ 새 고객 응답:")
    print()  # 빈 줄 추가
    
    # 타이핑 효과로 출력
    for char in response_text:
        print(char, end='', flush=True)
        time.sleep(0.03)
    
    print("\n")
    print("-" * 60)
    print("✅ 새 세션 확인 완료 - 이전 대화 맥락을 기억하지 않고 새로운 대화로 시작!")

except Exception as e:
    print(f"❌ 에러 발생: {e}")
    response = agentcore_runtime.invoke(
        {"prompt": user_query}, 
        bearer_token=bearer_token,
        session_id=str(session_id2)
    )
    
    if 'response' in response and isinstance(response['response'], str):
        response_text = response['response']
        if response_text.startswith('"') and response_text.endswith('"'):
            try:
                response_text = json.loads(response_text)
            except:
                pass
        print("✅ 새 고객 응답:")
        print(response_text)
    else:
        print("✅ 새 고객 응답:")
        print(response)

이 경우 에이전트는 더 이상 맥락을 가지고 있지 않으며 더 많은 정보가 필요합니다.

이것이 기반 인프라를 관리할 필요 없이 에이전트를 위한 안전하고 확장 가능한 엔드포인트를 갖는 데 필요한 모든 것입니다!

### Step 5: AgentCore 관측성

[AgentCore 관측성](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html)은 Amazon OpenTelemetry Python Instrumentation과 Amazon CloudWatch GenAI 관측성을 사용하여 AI 에이전트에 대한 모니터링 및 추적 기능을 제공합니다.

#### 에이전트

기본 AgentCore Runtime 구성은 **AgentCore 관측성**을 통해 CloudWatch에서 에이전트의 추적을 로깅할 수 있습니다. 이러한 추적은 AWS CloudWatch GenAI 관측성 대시보드에서 볼 수 있습니다. CloudWatch → GenAI 관측성 → Bedrock AgentCore로 이동하세요.

![CloudWatch의 에이전트 개요](images/observability_agents.png)

#### 세션

세션 보기는 계정의 모든 에이전트와 연결된 모든 세션 목록을 보여줍니다.

![세션](images/sessions_lab5_observability.png)

#### 추적

추적 보기는 이 계정의 에이전트에서 모든 추적을 나열합니다. 추적 작업:

- 특정 추적을 검색하려면 추적 필터를 선택하세요.
- 결과를 정리하려면 열 이름으로 정렬하세요.
- 작업에서 로그 인사이트를 선택하여 로그 및 스팬 데이터를 쿼리하여 검색을 개선하거나 선택된 추적 내보내기를 선택하여 내보내세요.

![추적](images/traces_lab4_observability.png)

### 🎉 축하합니다!

**Lab 4: 프로덕션 배포 - AgentCore Runtime과 관측성 사용**을 성공적으로 완료했습니다!

달성한 성과:

##### 프로덕션급 배포:

- 최소한의 코드 변경으로 에이전트를 프로덕션에 준비 (단 4줄만 추가)
- 서로 다른 고객 간 적절한 세션 격리 검증
- 세션별 연속성 + 메모리 지속성 및 컨텍스트 인식 확인

##### 엔터프라이즈급 보안 & 신원 관리:

- Cognito 통합을 통한 JWT 토큰 기반 보안 인증 구현
- 프로덕션 워크로드를 위한 적절한 IAM 역할 및 실행 권한 구성
- 보안 에이전트 호출을 위한 신원 기반 액세스 제어 설정

##### 포괄적인 관측성:

- 모든 고객 세션에서 완전한 요청 추적을 위한 AgentCore 관측성 활성화
- CloudWatch GenAI 관측성 대시보드 모니터링 구성

##### 현재 제한사항 (다음에 해결할 예정!):

- **개발자 중심 상호작용** - SDK/API 호출을 통한 에이전트 액세스이지만 사용자 친화적인 웹 인터페이스 없음
- **수동 세션 관리** - 직관적인 사용자 경험보다는 프로그래밍 방식의 세션 생성 필요

##### 다음 단계 [Lab 5: 사용자 인터페이스 구축 →](lab-05-frontend.ipynb)
Lab 5에서는 사용자 친화적인 인터페이스를 구축하여 고객 경험을 완성할 예정입니다! 계속 진행해보세요!!